In [ ]:
import time
import csv
from pathlib import Path
from skimage import io
import os
from artifact_removal.remove_hair import removeHair
from shared.features import  LesionFeatures

def preprocess_and_extract_features_in_memory(imageName, image=None, mask=None, doPlot = True):
    f1 = False if not image else removeHair(image, 6)

    featureStart = time.time()
    stats = LesionMaskStats(mask)
    asymmetryScore = asymmetry.getAsymmetryScore(stats)
    featureEnd = time.time()
    features = LesionFeatures(asymmetryScore)

    if doPlot & f1:
        plot_comparison.plotComparison(image, f1, f'Image {idx}')
    return f1, features, featureEnd - featureStart

def preprocess_and_extract_features_to_disk(imageName, image=None, mask=None, dist = 'dist', doPlot = True):
    results = preprocess_and_extract_features_in_memory(imageName, image, mask, doPlot)
    feat = results[1]
    Path(dist).mkdir(parents=True, exist_ok=True)
    Path(f'{dist}/images').mkdir(parents=True, exist_ok=True)
    if results[0]:
        io.imsave(f'{dist}/images/{fNameOnly}', results[0])

    if os.path.exists(f'{dist}/features.csv') == False:
        f = open(f'{dist}/features.csv', 'w+')
        writer = csv.writer(f, lineterminator='\n')
        writer.writerow(['Image'] + feat.featureNamesList + ['featureCalculationTime'])
        writer.writerow([imageName] + feat.featuresList() + [results[2]])
        f.close()
    else:
        f = open(f'{dist}/features.csv', 'a')
        writer = csv.writer(f, lineterminator='\n')
        writer.writerow([imageName] + feat.featuresList() + [results[2]])
        f.close()

from shared import (get_file_names, plot_comparison, read_image)
from feature_extraction import asymmetry
from feature_extraction.image_stats import LesionMaskStats

idx = 0
maskFilenames = get_file_names.getImagesInDir('/media/austin/Extreme 500/ISIC/masks', 'ISIC')
for f in maskFilenames:
    print('Processing image ', idx + 1, '/', len(maskFilenames))
    fPath = f.split('/')
    fNameOnly = fPath[-1]

    fNameNoExt = fNameOnly.split('.')[0]

    # maskPaths = fPath.copy()
    # maskName = fNameNoExt + '_mask.bmp'
    # maskPaths[-1] = maskName
    # maskPath = '/'.join(maskPaths)
    # print(f)
    # print(maskPath)


    #print('...reading image')
    image = read_image.readImage(f)
    #mask = read_image.readImage(maskPath)

    preprocess_and_extract_features_to_disk(fNameOnly, mask=image, dist='dist/02', )
    idx = idx + 1


Processing image  1 / 5932
Processing image  2 / 5932
Processing image  3 / 5932
Processing image  4 / 5932
Processing image  5 / 5932
Processing image  6 / 5932
Processing image  7 / 5932
Processing image  8 / 5932
Processing image  9 / 5932
Processing image  10 / 5932
Processing image  11 / 5932
Processing image  12 / 5932
Processing image  13 / 5932
Processing image  14 / 5932
Processing image  15 / 5932
Processing image  16 / 5932
Processing image  17 / 5932
Processing image  18 / 5932
Processing image  19 / 5932
Processing image  20 / 5932
Processing image  21 / 5932
Processing image  22 / 5932
Processing image  23 / 5932
Processing image  24 / 5932
Processing image  25 / 5932
Processing image  26 / 5932
Processing image  27 / 5932
Processing image  28 / 5932
Processing image  29 / 5932
Processing image  30 / 5932
Processing image  31 / 5932
Processing image  32 / 5932
Processing image  33 / 5932
Processing image  34 / 5932
Processing image  35 / 5932
Processing image  36 / 5932
P